# Three kinds of Implicits and Type Classes in Scala

blah blah blah

### -- Implicit Parameters --

Suppose you want a function that joins two strings together.

In [1]:
def example1(a: String, b: String): String = List(a, b).mkString("")

example1("foo", "bar")

defined function example1
res0_1: String = "foobar"

You can split the parameters into a parameter list as such.  Each set of parameters is called a parameter group.

In [2]:
def example2(a: String)(b: String): String = List(a, b).mkString("")

example2("foo")("bar")

defined function example2
res1_1: String = "foobar"

You can create a partially applied function (aka lambda) by passing only the first argument.  Then call that with the second argument.

In [3]:
val partial = example2("foo")(_)
partial("bar")

partial: String => String = ammonite.$sess.cmd2$Helper$$Lambda$2368/1130569246@2d9be5e5
res2_1: String = "foobar"

You can mark a parameter group within the parameter list as implicit.  This applies to all the parameters within that group.  When calling this function, all the non-implicit parameters must be supplied with arguments, as usual.  The implicit parameters can be supplied with arguments, if you want, but it is no longer required.  Instead, it will search for a variable, marked itself as implicit, that is in scope and of the matching type, for each of the parameters in the group.  If there are zero or more than one implicit variables in scope of the matching type, the code will not compile.

In [4]:
def example3(a: String)(implicit b: String): String = List(a, b).mkString("")

implicit val bar: String = "bar"

example3("foo")
example3("foo")("baz")

defined function example3
bar: String = "bar"
res3_2: String = "foobar"
res3_3: String = "foobaz"

blah blah blah

### -- Implicit Type Conversion --

blah blah blah

In [5]:
object Example4 {
  implicit def convert(x: Example5): Example4 = {
    val Array(firstName, lastName) = x.fullName.split(" ")
    Example4(firstName, lastName)
  }
}

case class Example4(firstName: String, lastName: String)

object Example5 {
  implicit def convert(x: Example4): Example5 = {
    val fullName = List(x.firstName, x.lastName).mkString(" ")
    Example5(fullName)
  }
}

case class Example5(fullName: String)

val a: Example4 = Example4("Bob", "Smith")
val b: Example5 = a
val c: Example4 = b

defined object Example4
defined class Example4
defined object Example5
defined class Example5
a: Example4 = Example4("Bob", "Smith")
b: Example5 = Example5("Bob Smith")
c: Example4 = Example4("Bob", "Smith")

Scala has an objected called `Predef`, that is automatically imported into every file.  There are all kinds of interesting things in there.  And it includes several implicit type conversions.  For example, anywhere a `Double` is required, you can provide an `Int`, which will get automatically converted to the right type.  This makes sense, because `Int` to `Double` does not loose any detail.  But detail would be lost going the opposite direction, which is why that will not compile.  You would have to explicitly make the conversion in whatever manner makes sense for your applicaiton.

In [6]:
val d: Int = 123
val e: Double = d

d: Int = 123
e: Double = 123.0

### -- Implicit Methods --

Consider calling the method `distinct` on a `String`.

In [7]:
"aaabbbccc".distinct

res6: String = "abc"

In Scala, the `String` type is an alias for the Java `String`.  And the `String` class in Java does not have a `distinct` method as a member.  So what is going on here?  Where did this method come from?  It came from the `StringOps` class in Scala.

If you have a variable of type A and try to call a method that is not a member of the class, it will try to find a single implicit function within scope that converts the variable from type A to any type B which does have the method as a member.  If there are zero or more than one type conversion functions within scope which meet this criteria, the code will not compile. 

Within the `Predef` object, there is an implicit function called `augmentString`, which converts a `String` to a `StringOps`.  Thus the `String` is automatically converted into a `StringOps` because it has the `distinct` method as a member.

In [8]:
case class MyStringOps(string: String) {
  def multiply1(x: Int): String = (1 to x).map(_ => string).mkString("")
}

implicit def convert1(string: String): MyStringOps = MyStringOps(string)

"hello".multiply1(3)

defined class MyStringOps
defined function convert1
res7_2: String = "hellohellohello"

This kind of implicit and it's behavior is the required basis for type classes described next.

### -- Type Classes --

Type classes are an ad-hoc method of adding behaviors to a set of classes.  The concept came from __[Haskell](http://learnyouahaskell.com/types-and-typeclasses#typeclasses-101)__.  The first step is to create a `trait` which defines the methods and type signatures for the desired behaviors.  Type classes are also the foundation for functional programming libraries such as __[Cats](https://typelevel.org/cats/)__ and __[Scalaz](https://scalaz.github.io/)__.

In [9]:
trait MyTypeClass[A] {
  def multiply2(x: Int): A
}

defined trait MyTypeClass

The next step is to create implicit type conversion functions from each of the classes you want to have this set of behaviors to an annonymous implementation of the trait.

In [10]:
implicit def convert2(string: String) = new MyTypeClass[String] {
  def multiply2(x: Int): String = (1 to x).map(_ => string).mkString("")
}

implicit def convert3(int: Int) = new MyTypeClass[Int] {
  def multiply2(x: Int): Int = (1 to x).map(_ => int.toString).mkString("").toInt
}

defined function convert2
defined function convert3

Then you can simply call any of the trait's methods on any of the classes belonging to the type class.

In [11]:
"hello".multiply2(3)
123.multiply2(3)

res10_0: String = "hellohellohello"
res10_1: Int = 123123123